In [162]:
import pandas as pd
import numpy as np
import seaborn as sns

In [163]:
data = pd.read_csv('x17.data', sep='  ',engine='python')
data.drop(['Index'],axis=1,inplace=True)
data

,Material_1_amount,Material_2_amount,Material_3_amount,Condition,Octane_number
0,55.33,1.72,54,1.66219,92.19
1,59.13,1.20,53,1.58399,92.74
2,57.39,1.42,55,1.61731,91.88
3,56.43,1.78,55,1.66228,92.80
4,55.98,1.58,54,1.63195,92.56
...,...,...,...,...,...
77,67.60,1.84,55,1.64758,91.86
78,64.81,2.24,54,1.69592,91.61
79,63.13,1.60,52,1.66118,92.17
80,63.48,3.46,52,1.48216,91.56


In [164]:
data.corr() #Launch_temperature affects thermal distress the most!!

,Material_1_amount,Material_2_amount,Material_3_amount,Condition,Octane_number
Material_1_amount,1.000000,-0.589451,0.448733,-0.336917,-0.870159
Material_2_amount,-0.589451,1.000000,-0.298396,0.161196,0.392359
Material_3_amount,0.448733,-0.298396,1.000000,-0.721748,-0.638490
Condition,-0.336917,0.161196,-0.721748,1.000000,0.628730
Octane_number,-0.870159,0.392359,-0.638490,0.628730,1.000000


In [165]:
len(data)

82

In [166]:
data.drop(data.columns[[1,2,3]],axis=1) #check with condition also

,Material_1_amount,Octane_number
0,55.33,92.19
1,59.13,92.74
2,57.39,91.88
3,56.43,92.80
4,55.98,92.56
...,...,...
77,67.60,91.86
78,64.81,91.61
79,63.13,92.17
80,63.48,91.56


In [167]:
def norm(numpy_arr,a,b):
    numpy_arr=(numpy_arr-a)/(b-a)
    return numpy_arr

In [168]:
y=data["Octane_number"].tolist()
op_max=max(y);op_min=min(y)
x=data["Material_1_amount"].tolist()
ip_max=max(x);ip_min=min(x)

norm_x=norm(np.array(x),ip_min,ip_max);

norm_y=norm(np.array(y),op_min,op_max);


outputs=norm_x[:57];y_test=norm_y[57:]
inputs=norm_x[:57];x_test=norm_y[57:]

In [169]:
y_test

array([0.02641509, 0.04025157, 0.0427673 , 0.        , 0.05283019,
       0.12704403, 0.11698113, 0.17610063, 0.12955975, 0.18238994,
       0.08301887, 0.08805031, 0.11446541, 0.5672956 , 0.59496855,
       0.47295597, 0.62767296, 1.        , 0.93333333, 0.68679245,
       0.27672956, 0.24528302, 0.31572327, 0.23899371, 0.31446541])

In [175]:
w1=2.1;w2=-1.6;w3=0.36;w4=0.63
b1=0;b2=0;b3=0;ssr=9223372036854775807

In [176]:
while ssr>0.1:

    #network
    z1=w1*inputs+b1
    z2=w2*inputs+b2
    a1=np.log(1+(np.e)**z1)
    a2=np.log(1+(np.e)**z2)
    a3=w3*a1;a4=w4*a2
    pred=a3+a4+b3
    z1=[0]*len(inputs);z2=[0]*len(inputs);a1=[0]*len(inputs);a2=[0]*len(inputs);
    a3=[0]*len(inputs);a4=[0]*len(inputs);pred=[0]*len(inputs);
    
    #backpropagation
    der_w1=0;der_b1=0;der_w2=0;der_b2=0;der_w3=0;der_w4=0;der_b3=0
    ssr=0
    for i in range(len(inputs)):
        z1[i]=w1*inputs[i]+b1
        z2[i]=w2*inputs[i]+b2
        a1[i]=np.log(1+(np.e)**z1[i])
        a2[i]=np.log(1+(np.e)**z2[i])
        a3[i]=w3*a1[i];a4[i]=w4*a2[i]
        pred[i]=a3[i]+a4[i]+b3
        
        der_w1+=-2*w3*(outputs[i]-pred[i])*inputs[i]/(1+(np.e)**(-z1[i]))
        der_b1+=-2*w3*(outputs[i]-pred[i])/(1+(np.e)**(-z1[i]))
        der_w2+=-2*w3*(outputs[i]-pred[i])*inputs[i]/(1+(np.e)**(-z2[i]))
        der_b2+=-2*w3*(outputs[i]-pred[i])/(1+(np.e)**(-z2[i]))
        der_w3+=-2*a1[i]*(outputs[i]-pred[i])
        der_w4+=-2*a2[i]*(outputs[i]-pred[i])
        der_b3+=-2*(outputs[i]-pred[i])
        ssr+=(outputs[i]-pred[i])**2
        
    print(ssr)
    w1=w1-0.01*der_w1
    b1=b1-0.01*der_b1
    w2=w2-0.01*der_w2
    b2=b2-0.01*der_b2
    w3=w3-0.01*der_w3
    w4=w4-0.01*der_w4
    b3=b3-0.01*der_b3

0.17652285685465124
0.6626235880352528
9.699873023998485
154.50444696020503
676.3756837969844
599.6131698236503
12.18158408147012
0.6542032154207523
0.4280370233493545
0.4235948973243179
0.42350299196142493
0.4234964312063812
0.4234915384342465
0.42348667188260963
0.42348179933295804
0.4234769201434567
0.423472034292161
0.4234671417692219
0.4234622425650169
0.42345733666991353
0.4234524240742615
0.42344750476839443
0.4234425787426295
0.42343764598726863
0.4234327064925953
0.42342776024887785
0.4234228072463674
0.4234178474752998
0.4234128809258925
0.4234079075883472
0.4234029274528495
0.42339794050956664
0.4233929467486511
0.4233879461602374
0.4233829387344436
0.42337792446137046
0.4233729033311027
0.4233678753337079
0.42336284045923606
0.4233577986977213
0.42335275003917916
0.4233476944736106
0.4233426319909969
0.42333756258130345
0.4233324862344787
0.4233274029404542
0.42332231268914305
0.42331721547044177
0.4233121112742308
0.4233070000903705
0.42330188190870693
0.4232967567190666
0

0.42042589701627264
0.42041672363154226
0.420407536956933
0.42039833697072954
0.42038912365117587
0.4203798969764726
0.4203706569247791
0.4203614034742118
0.42035213660284454
0.42034285628870854
0.42033356250979315
0.420324255244044
0.42031493446936424
0.42030560016361457
0.4202962523046117
0.42028689087013005
0.42027751583789963
0.42026812718560885
0.42025872489090127
0.42024930893137685
0.4202398792845933
0.4202304359280635
0.4202209788392562
0.4202115079955973
0.4202020233744675
0.42019252495320447
0.4201830127091007
0.42017348661940535
0.42016394666132184
0.42015439281201017
0.42014482504858536
0.4201352433481174
0.4201256476876319
0.42011603804410935
0.42010641439448476
0.4200967767156488
0.42008712498444617
0.42007745917767664
0.42006777927209454
0.4200580852444081
0.4200483770712804
0.4200386547293296
0.4200289181951254
0.4200191674451942
0.420009402456015
0.41999962320402073
0.4199898296655986
0.4199800218170887
0.4199701996347849
0.4199603630949347
0.4199505121737384
0.4199406

0.41304870560589335
0.41302809007710756
0.4130074405994334
0.4129867571067797
0.41296603953290334
0.4129452878114072
0.41292450187574253
0.4129036816592061
0.41288282709494084
0.41286193811593497
0.41284101465502276
0.41282005664488236
0.4127990640180372
0.4127780367068538
0.41275697464354255
0.41273587776015735
0.41271474598859476
0.41269357926059286
0.4126723775077322
0.4126511406614352
0.4126298686529649
0.4126085614134246
0.4125872188737583
0.41256584096475013
0.4125444276170227
0.41252297876103816
0.41250149432709693
0.41247997424533756
0.4124584184457364
0.41243682685810645
0.4124151994120986
0.41239353603719886
0.4123718366627294
0.41235010121784926
0.41232832963155036
0.41230652183266103
0.41228467774984245
0.4122627973115902
0.41224088044623314
0.41221892708193253
0.4121969371466828
0.4121749105683092
0.4121528472744699
0.41213074719265247
0.4121086102501765
0.41208643637419023
0.41206422549167376
0.4120419775294337
0.4120196924141078
0.4119973700721609
0.41197501042988616
0.4

0.39054569052571897
0.3904831622043959
0.39042050660170197
0.3903577234109809
0.3902948123247522
0.39023177303470624
0.39016860523170127
0.3901053086057639
0.3900418828460849
0.38997832764101736
0.3899146426780741
0.3898508276439265
0.38978688222440017
0.3897228061044755
0.3896585989682825
0.3895942604990997
0.38952979037935287
0.38946518829061066
0.38940045391358413
0.3893355869281231
0.38927058701321443
0.38920545384698024
0.389140187106673
0.3890747864686777
0.38900925160850514
0.3889435822007914
0.3888777779192963
0.38881183843689904
0.3887457634255981
0.38867955255650644
0.3886132054998515
0.38854672192497086
0.38848010150031104
0.388413343893425
0.3883464487709684
0.3882794157986995
0.3882122446414749
0.38814493496324776
0.38807748642706535
0.3880098986950668
0.3879421714284809
0.38787430428762204
0.3878062969318903
0.38773814901976683
0.38766986020881344
0.38760143015566756
0.38753285851604136
0.3874641449447209
0.3873952890955601
0.3873262906214803
0.38725714917446924
0.3871878

0.3321970481513287
0.3320043505417291
0.3318112102908414
0.3316176264385849
0.3314235980242928
0.3312291240867242
0.33103420366408187
0.33083883579402945
0.33064301951370667
0.3304467538597465
0.3302500378682918
0.3300528705750135
0.3298552510151263
0.329657178223407
0.3294586512342124
0.32925966908149606
0.32906023079882757
0.32886033541940957
0.3286599819760963
0.3284591695014139
0.3282578970275757
0.32805616358650497
0.3278539682098518
0.3276513099290116
0.3274481877751481
0.32724460077920914
0.32704054797194904
0.3268360283839484
0.3266310410456328
0.3264255849872961
0.3262196592391185
0.3260132628311887
0.32580639479352597
0.3255990541560988
0.3253912399488496
0.3251829512017134
0.3249741869446433
0.3247649462076284
0.3245552280207208
0.3243450314140539
0.32413435541786906
0.3239231990625357
0.32371156137857554
0.3234994413966874
0.3232868381477682
0.32307375066293903
0.3228601779735697
0.3226461191113004
0.32243157310806925
0.3222165389961351
0.3220010158081037
0.3217850025769531

0.14274336378722088
0.14230394514284259
0.1418647402513244
0.1414257542738407
0.14098699237375364
0.1405484597163583
0.1401101614686248
0.13967210279894124
0.13923428887685368
0.13879672487280614
0.13835941595788
0.1379223673035328
0.13748558408133463
0.137049071462706
0.1366128346186532
0.13617687871950354
0.13574120893464012
0.13530583043223532
0.13487074837898486
0.1344359679398395
0.13400149427773725
0.13356733255333492
0.13313348792473964
0.13269996554723765
0.13226677057302635
0.1318339081509418
0.13140138342618896
0.13096920154006947
0.13053736762971174
0.1301058868277967
0.12967476426228686
0.12924400505615205
0.12881361432709998
0.12838359718729922
0.12795395874310855
0.12752470409480288
0.12709583833629856
0.12666736655488223
0.12623929383093446
0.12581162523765865
0.12538436584080478
0.12495752069839795
0.12453109486046272
0.1241050933687525
0.12367952125647269
0.12325438354801047
0.12282968525865952
0.12240543139434855
0.12198162695136945
0.121558276916102
0.121135386264745

In [177]:
print(b3)

1.0373351794474133


In [178]:
def rev_norm(array,a,b):
    for i in range(len(array)):
        array[i]=a+array[i]*(b-a)
    return array

In [179]:
#testing
z1_test=[0]*len(x_test);z2_test=[0]*len(x_test);a1_test=[0]*len(x_test);a2_test=[0]*len(x_test);
a3_test=[0]*len(x_test);a4_test=[0]*len(x_test);pred_test=[0]*len(x_test);
mse=0
for i in range(len(x_test)):
        z1_test[i]=w1*x_test[i]+b1
        z2_test[i]=w2*x_test[i]+b2
        a1_test[i]=np.log(1+(np.e)**z1_test[i])
        a2_test[i]=np.log(1+(np.e)**z2_test[i])
        a3_test[i]=w3*a1_test[i];a4_test[i]=w4*a2_test[i]
        pred_test[i]=a3_test[i]+a4_test[i]+b3

#pred_revnorm=rev_norm(pred_test,op_min,op_max)
#outputs_test=y[57:]
for i in range(len(x_test)):
    mse+=(y_test[i]-pred_test[i])**2

mse=mse/len(x_test)    
mse=mse**0.5
print(mse)

0.06538400119171259
